## Flowpass

Todo...
Audio Core:
    - Pass state and queue through cffi for GIL-free callback
    - Add a lot more modules
Thread Interface:
    - Generalize the queue data protocol for any control params
    - Add time scheduling
    - Create way to set the module program
Live Sequencer:
    - Add note duration control
    

In [12]:
# %load RealTimeTest.py

from sounddevice import Stream, CallbackStop
import time
from numpy import array, random, zeros
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import math
import sounddevice as sd
import threading
from IPython.display import display, clear_output
import ipywidgets as widgets
import queue
import pdb

audio = None
core = None
gui = None
p = None

In [50]:


class Audio:
    def __init__(self):
        self.alive = True
        self.endevent = threading.Event()
        self.startevent = threading.Event()
        self.thread = threading.Thread(target=self.rebootloop)
        self.thread.start()
        
    def callback(self, indata, outdata, frames, time, status):
        global core
#         if status:
#             print(status)
        if not self.playing:
            raise sd.CallbackAbort
    #     print(time.outputBufferDacTime)
        audiomain(indata, outdata, frames, time, core)

    def rebootloop(self):
        while self.alive:
            self.startevent.wait()
            self.startevent.clear()
            if not self.alive:
                break
            with sd.Stream(channels = 2, 
                           callback = self.callback, 
                           blocksize = 256,
                           finished_callback = self.endevent.set):
                self.endevent.wait()
            self.endevent.clear()
            
    def start(self):
        self.playing = True
        self.startevent.set()
        print('start')
        
    def stop(self):
        self.playing = False
        self.endevent.set()
        print('stop')
        
    def kill(self):
        self.alive = False
        self.start()
        self.stop()
        

try:
    audio.kill()
except:
    pass

audio = Audio()



start
stop


In [59]:

# @jit(nopython=False)
def audiomain(indata, outdata, frames, time, core):
    if not core.program or not core.outputs:
        return
    for x in core.program:
#         x[3][:] *= 0
        x[0](x[1], x[2], x[3], frames)
        
    outdata[:frames,:] = np.zeros((frames,2))
    for x in core.outputs:
        outdata[:frames,0] += x[:frames]
        outdata[:frames,1] += x[:frames]
    
class AudioCore:
    def __init__(self):
        self.queues = [queue.Queue(128)]*32
        self.states = np.zeros(64*1024)
        self.buffers = np.zeros((32,256))
        self.program = []
        self.outputs = []
        
core = AudioCore()

class CoreBuilder:
    def __init__(self):
        pass
    

In [95]:


@jit(nopython=True)
def ak_xwave_kernel(ins, states, outs, frames):
    freq = ins
    p0,fz1 = states[0:2]
    output,phase = outs
    
    for i in range(frames):
        phase[i] = p0
        p0 = p0 + freq[i]/44000
        
    for j in range(1,40):
        output[:] += 0.1 * np.sin(phase[:] * math.pi * 2 * j) / j
        
    for i in range(frames):
        fz1 = fz1 * 0.9 + output[i] * 0.1
        output[i] = fz1
        
    states[0:2] = p0,fz1


# ak_xwave = {
#     'inputs': 1,
#     'states': 2,
#     'outputs': 2,
#     'kernel': ak_xwave_kernel,
# }


@jit(nopython=True)
def supersaw_kernel(ins, states, outs, frames):
    freq,gate = ins
    output,tmp = outs
    
    output[:frames] = np.zeros(frames)
    
    ak_xwave_kernel((freq), states[0:2], outs, frames)
    
    output *= gate


def player_kernel(q, states, outs, frames):
    freq,gate = outs
    f0,ttl = states
    
    for i in range(16):
        try:
            e = q.get_nowait()
            if e[0] == 0xa:
                gate[:] = np.ones(frames)
#                 print(e[1])
                f0 = e[1]
                ttl = 0.5
        except:
            break
    ttl -= frames/44100
    
    freq[:frames] = np.ones(frames) * f0
    gate[:frames] = np.ones(frames) * int(ttl > 0)
    
    states[:] = f0,ttl

core.program = [
    (player_kernel, core.queues[0], core.states[2:4], (core.buffers[1],core.buffers[2])),
    (supersaw_kernel, (core.buffers[1],core.buffers[2]), core.states[0:2], (core.buffers[3],core.buffers[4])),
]
core.outputs = [core.buffers[3]]



In [38]:

class Gui:
    def __init__(self):
        self.playb = widgets.Button(description="play")
        self.stopb = widgets.Button(description="stop")

        self.playb.on_click(self.play_click)
        self.stopb.on_click(self.stop_click)
        self.reoutput()
    
    def play_click(self,b):
        self.reoutput()
        audio.start()

    def stop_click(self,b):
        audio.stop()

    def reoutput(self):
        clear_output(wait=True)
        display(self.playb, self.stopb)

gui = Gui()

Button(description='play', style=ButtonStyle())

Button(description='stop', style=ButtonStyle())

start


In [94]:
import time

class Pattern:
    def __init__(self, data, mod=1, speed=1):
#         self.curs = 0
        self.source = data
        self.data = data
        self.mod = mod * len(self.data)
        self.speed = speed
        for i in range(len(data)):
            if isinstance(data[i], list):
                self.data[i] = Pattern(self.data[i], self.mod)
    def __repr__(self):
        return 'p(' + str(self.data) + ')'
    def next(self, beat):
        c = int((beat % self.mod) / (self.mod / len(self.data)))
        r = self.data[c]
        if isinstance(r, Pattern):
            r = r.next(beat)
#         self.curs = (self.curs+1) % len(self.data)
        return r
    def __mul__(self, x):
        if isinstance(x, (int, float)):
            self.speed *= x
        return self
            
    
class Player:
    def __init__(self):
        self.pat = None
        self.oct = 2
    def __rrshift__(self, data):
        if data is None:
            self.pat = None
            return
        if isinstance(data, str) or isinstance(data, list):
            data = Pattern(data)
        if isinstance(data, Pattern):
            self.pat = data
        else:
            raise TypeError
        return self
    def active(self):
        return self.pat != None
    def beat(self, beat):
        n = self.pat.next(beat)
        freq = p.freq(n) * 2**self.oct
        global core
        try:
            core.queues[0].put((0xa, freq))
#             print('put',freq)
        except:
            pass
    
class Synth:
    def __init__(self):
        pass
    
class LiveEnv:
    def __init__(self):
        self.alive = True
        self.players = [None]*32
        self.clock = 0
        self.lastbeat = -1
        self.lastpoll = None
        
        self.bpm = 140
        self.basefreq = 27.5
        self.scale = [0,2,3,5,7,8,10]
        
        self.pollthread = threading.Thread(target=self._seqloop)
        self.pollthread.start()
        
    def __call__(self, thing):
        if isinstance(stream, str) or isinstance(stream, list):
            return Pattern(stream)
    def __getitem__(self, index):
        if self.players[index] is None:
            self.players[index] = Player()
        return self.players[index]
    def freq(self, note):
        halfsteps = self.scale[note % len(self.scale)]
#         print(halfsteps)
        halfsteps += 12 * (note // len(self.scale))
        return self.basefreq * math.exp(math.log(2)/12 * halfsteps)
    def _seqloop(self):
        while self.alive:
            self.seqpoll()
            time.sleep(0.01)
    def active(self):
        for p in self.players:
            if p and p.active():
                return True
        return False
    def seqpoll(self):
        curpoll = time.time()
        if self.lastpoll and self.active():
            dt = curpoll - self.lastpoll
            self.clock += dt / (60. / self.bpm)
            beat = math.floor(self.clock)
            if beat != self.lastbeat:
                for p in self.players:
                    if p and p.active():
                        p.beat(beat)
            self.lastbeat = beat
        self.lastpoll = curpoll
    
    def refresh(self):
        pass
        
    def kill(self):
        self.alive = False

    
try:
    p.kill()
except:
    pass

p = LiveEnv()


In [93]:
[0,[5,4,5,6,7,8,9,8]] >> p[1] #>> supersaw
[0,[5,4,5,6,7,8,9,8]] >> p[1] #>> supersaw
p.bpm = 140
# None >> p[1]

In [ ]:
p.refresh()
p.scale = 'C# minor'
p.bpm = 140
'C# minor' >> p
'120' >> p
[0,1,2,3] >> p[1] >> supersaw
'-xoo' >> p[2] >> drum1

In [ ]:
[1,2,3] >> P[0]
P[0].pat

In [9]:
x = np.zeros(3)
y=x[1:3]
a,b = y
a = 1
b = 2
y[:] = b,a
x

array([ 0.,  2.,  1.])